# Baseline Analysis: LV Feeder

This notebook establishes the **baseline operating condition** of a CSIRO low-voltage (LV) radial feeder before introducing EVs, batteries, or D-Suite devices.

The baseline is used as a **reference case** to understand:
- existing voltage behaviour,
- loading patterns,
- and constraint activation under normal demand.

All future scenarios are compared back to this baseline.

This collection was used:

> Heidarihaei, Rahmatollah; Geth, Frederik; & Sander, Claeys (2024): Four-wire low voltage power network dataset. v1. CSIRO. Data Collection. 
> https://doi.org/10.25919/jaae-vc35


### Workflow overview

This baseline analysis follows three steps:

1. **Identify the correct CSIRO LV feeder**
   - Screen available networks in `00_*.ipynb`
   - Select a representative radial LV feeder

2. **Load the feeder**
   - Implement a dedicated loader function in `network_loader.jl`

3. **Solve and analyse the baseline**
   - Run power flow / OPF with no EVs or D-Suite devices
   - Plot voltage profiles along the feeder
   - Check compliance with AS 60038 voltage limits.


## 4-Wire Feeder 

### Selected feeder

The feeder used in this study was selected by screening the CSIRO four-wire LV dataset to identify a **small, interpretable, radial network** suitable for intuition-building.

Selection criteria:
- Four-wire representation
- Radial topology
- Fewer than 100 buses
- Moderate number of customer loads

Chosen feeder:
- Dataset: CSIRO four-wire low voltage power network dataset
- Network: network_20
- Feeder: Feeder_5
- Size: ~65 buses, ~23 loads

This feeder is small enough to visualise clearly, while still exhibiting realistic voltage drop behaviour.


In [ ]:
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()   
using PowerModelsDistribution
using DataFrames
using Ipopt
using Plots
using JuMP
using LinearAlgebra

  Activating project at `/mnt/c/Users/auc009/OneDrive - CSIRO/Documents/power-models-distribution/pmd_ev_experiments`


### Parsing the Feeder 

In [16]:
file = "../data/networks/lv_small_feeder/Master.dss"

eng4w = parse_file(file, transformations=[transform_loops!])

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 23 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 24 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 25 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 31 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line48 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : ba

Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "enwl_network_20_feeder_5_4wire"
  "line"           => Dict{String, Any}("line48"=>Dict{String, Any}("length"=>8…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>100000.0, "vbases_defa…
  "files"          => ["../data/networks/lv_small_feeder/Master.dss", "../data/…
  "load"           => Dict{String, Any}("load2"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("32"=>Dict{String, Any}("rg"=>Float64[]…
  "linecode"       => Dict{String, Any}("lc8"=>Dict{String, Any}("b_fr"=>[0.0 0…
  "data_model"     => ENGINEERING

In [25]:
println("Number of buses in eng4w: ", length(get(eng4w, "bus", Dict())))
println("Number of lines in eng4w: ", length(get(eng4w, "line", Dict())))

Number of buses in eng4w: 65
Number of lines in eng4w: 64


In [20]:
eng4w["voltage_source"]["source"]

Dict{String, Any} with 9 entries:
  "source_id"     => "vsource.source"
  "rs"            => [0.000642012 5.94953e-5 5.94953e-5 5.94953e-5; 5.94953e-5 …
  "va"            => [0.0, -120.0, 120.0, 0.0]
  "status"        => ENABLED
  "connections"   => [1, 2, 3, 4]
  "vm"            => [0.240178, 0.240178, 0.240178, 0.0]
  "xs"            => [0.00231438 -1.56862e-5 -1.56862e-5 -1.56862e-5; -1.56862e…
  "bus"           => "sourcebus"
  "configuration" => WYE

In [ ]:
math4w = transform_data_model(eng4w; kron_reduce=false, phase_project=false);
sourcebus = math4w["bus_lookup"]["sourcebus"]
math4w["bus"]["$sourcebus"]

Dict{String, Any} with 12 entries:
  "vm_pair_lb" => Tuple{Any, Any, Real}[]
  "grounded"   => Bool[0, 0, 0, 1]
  "vm_pair_ub" => Tuple{Any, Any, Real}[]
  "bus_i"      => 60
  "name"       => "sourcebus"
  "bus_type"   => 1
  "terminals"  => [1, 2, 3, 4]
  "vmax"       => [Inf, Inf, Inf, Inf]
  "vbase"      => 0.240178
  "source_id"  => "bus.sourcebus"
  "vmin"       => [0.0, 0.0, 0.0, 0.0]
  "index"      => 60

In [32]:
keys(math4w)

KeySet for a Dict{String, Any} with 18 entries. Keys:
  "is_kron_reduced"
  "conductor_ids"
  "time_elapsed"
  "bus"
  "name"
  "map"
  "settings"
  "gen"
  "branch"
  "storage"
  "switch"
  "is_projected"
  "per_unit"
  "data_model"
  "shunt"
  "transformer"
  "bus_lookup"
  "load"

In [34]:
println("Running unbalanced AC Optimal Power Flow with Ipopt...")

result_opf = solve_mc_opf(
    math4w,          
    IVRENPowerModel,     
    Ipopt.Optimizer
)

println("OPF solve status: ", result_opf["termination_status"])
println("Objective value (if any): ", get(result_opf, "objective", missing))

Running unbalanced AC Optimal Power Flow with Ipopt...
This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:     8688
Number of nonzeros in inequality constraint Jacobian.:     1036
Number of nonzeros in Lagrangian Hessian.............:     1974

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total seconds in IPOPT                               = 0.077

EXIT: Invalid number in NLP function or derivative detected.
OPF solve status: INVALID_MODEL
Objective value (if any): 0

### Plotting

In [28]:
function get_bus_name_phases(bus_name)
    bus_split = split(bus_name, ".")
    bus = bus_split[1]
    bus_phases = parse.(Int64, bus_split[2:end])

    return (bus, bus_phases)
end

get_bus_name_phases (generic function with 1 method)

In [ ]:
function plot_voltage_along_feeder_snap(buses_dict, lines_df; t=1, Vthreshold=1000, vmin = 0.94*230, vmax = 1.1*230)
    # plot(1:10)
    p = plot(legend=false)
    ylabel!("Voltage magnitude P-N (V)")
    title!("Voltage drop along feeder")
    xlabel!("Distance from reference bus (km)")
    colors = [:blue, :red, :black]
    for line in DataFrames.eachrow(lines_df)
        (bus1_name, bus1_phases) = get_bus_name_phases(line.Bus1)
        (bus2_name, bus2_phases) = get_bus_name_phases(line.Bus2)
        @assert length(bus1_phases) == length(bus2_phases)
        for c in 1:length(bus1_phases)
            dist_f_bus = buses_dict[bus1_name]["distance"]
            dist_t_bus = buses_dict[bus2_name]["distance"]
            phase = bus1_phases[c]
            vm_f = 1000
            vm_t = 1000
            if phase == 1
                vm_f = buses_dict[bus1_name]["vma"][t]
                vm_t = buses_dict[bus2_name]["vma"][t]
            elseif phase == 2
                vm_f = buses_dict[bus1_name]["vmb"][t]
                vm_t = buses_dict[bus2_name]["vmb"][t]
            elseif phase == 3
                vm_f = buses_dict[bus1_name]["vmc"][t]
                vm_t = buses_dict[bus2_name]["vmc"][t]
            end
            if vm_f < Vthreshold && vm_t < Vthreshold
                plot!([dist_f_bus; dist_t_bus], [vm_f; vm_t], color=colors[phase], marker=:circle, markersize=1)
            end
        end
    end


# Baseline visualisation 

These plots are used for **qualitative comparison** with EV and D-Suite scenarios.


In [ ]:
using PowerModelsDistribution
using PowerPlots
using DataFrames
using Graphs
using CairoMakie
using GraphMakie
using Graphs


In [ ]:
keys(pm)


In [ ]:
first(values(pm["bus"])) |> keys
bus_ids = collect(keys(pm["bus"]))

In [ ]:
g = Graph()

### Network Graph

In [ ]:
bus_ids = collect(keys(pm["bus"]))

for b in bus_ids
    add_vertex!(g)
end


In [ ]:
bus_index = Dict(bus_ids[i] => i for i in eachindex(bus_ids))

In [ ]:
for (_, line) in pm["line"]
    f = bus_index[line["f_bus"]]
    t = bus_index[line["t_bus"]]
    add_edge!(g, f, t)
end


In [ ]:
f = Figure(size = (800, 600))
ax = Axis(f[1,1], title = "LV Network Topology (Connectivity Only)")

graphplot!(
    ax,
    g,
    node_size = 10,
    node_color = :black,
    edge_color = :gray
)

f
save(joinpath(FIG_DIR, "csiro_small_feeder_topology.png"), f)

In [ ]:
f

### LV Network Topology (Source Bus Highlighted)

In [ ]:
slack_id = "sourcebus"


In [ ]:
bus_ids = collect(keys(pm["bus"]))
bus_index = Dict(bus_ids[i] => i for i in eachindex(bus_ids))


In [ ]:
slack_idx = bus_index[slack_id]


In [ ]:
node_colors = fill(:gray, nv(g))
node_colors[slack_idx] = :red

f1 = Figure(size = (800, 600))
ax = Axis(f1[1,1], title = "LV Network Topology (Source Bus Highlighted)")

graphplot!(
    ax,
    g,
    node_size = 10,
    node_color = node_colors,
    edge_color = :black
)

f1
save(joinpath(FIG_DIR, "csiro_small_feeder_topology_sourcebus.png"), f1)

In [ ]:
f1

### LV Network Topology (with Electrical distance from Source Bus)

In [ ]:
dist = dijkstra_shortest_paths(g, slack_idx).dists


In [ ]:
dist_norm = dist ./ maximum(dist)


In [ ]:
node_colors = fill(:gray, nv(g))
node_colors[slack_idx] = :red

f2 = Figure(size = (800, 600))
ax = Axis(f2[1,1], title = "LV Network Topology (with Electrical distance from Source Bus)")

graphplot!(
    ax,
    g,
    node_size = 10,
    node_color = dist_norm,
    colormap = :viridis,
    edge_color = :gray
)

f2
save(joinpath(FIG_DIR, "csiro_small_feeder_topology_distance_sourcebus.png"), f2)






In [ ]:
f2

### LV Network Topology – Baseline Voltage Profile

In [ ]:
using Statistics

df_bus_vm = combine(
    groupby(df_v, :bus),
    :vm_pu => mean => :vm_mean
)


In [ ]:
vm_map = Dict(row.bus => row.vm_mean for row in eachrow(df_bus_vm))

vm_vec = [
    vm_map[bus_id] for bus_id in bus_ids
]


In [ ]:
using CairoMakie
using GraphMakie

f3 = Figure(size = (900, 700))
ax = Axis(
    f3[1,1],
    title = "LV Network Topology – Baseline Voltage Profile",
    xlabel = "",
    ylabel = ""
)

graphplot!(
    ax,
    g,
    node_color = vm_vec,
    colormap = :plasma,
    node_size = 12,
    edge_color = :gray
)

Colorbar(f3[1,2], limits = extrema(vm_vec), label = "Voltage (p.u.)")

f3
save(joinpath(FIG_DIR, "csiro_small_feeder_topology_voltage_profile.png"), f3)

In [ ]:
f3

### Daisy plot

In [ ]:
dist = dijkstra_shortest_paths(g, slack_idx).dists


In [ ]:
scatter(
    dist,
    vm_vec,
    xlabel = "Electrical Distance from Transformer",
    ylabel = "Voltage (p.u.)",
    title = "Baseline Voltage Profile Along Feeder",
    markersize = 10
)
hline!([V_MIN, V_MAX], linestyle = :dash)

